<a href="https://colab.research.google.com/github/elisafm4/API_REST_course/blob/master/FLEXCONN_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Procesado Imágenes

Cargamos imágenes y hacemos preprocesado (a nivel local).

Obtenemos el directorio de las imágenes y el número de pacientes (N).

In [0]:
import os
import re
import numpy as np

dir_name = "LS_FLEXCONN"
atlas = os.listdir(dir_name)

nums = []

for im in atlas:
    nums.append(re.findall('\d+', im)[0])

N = np.unique(nums)

Para cada paciente, aplicamos un skull-stripping para eliminar lo que no pertenece al cerebro tanto en T1 como en FLAIR y reducir posibles falsos positivos. Los nuevos volúmenes junto con el volumen de las lesiones son guardados en un nuevo directorio 'Atlas'.

In [0]:
import nibabel as nib
import shutil

for i in N:
    volT1 = nib.load("{}/atlas{}_T1.nii.gz".format(dir_name, i))
    volFL = nib.load("{}/atlas{}_FL.nii.gz".format(dir_name, i))
    mask = nib.load("{}/atlas{}_T1_mask.nii.gz".format(dir_name, i)).get_fdata()
    T1_pre = volT1.get_fdata() * mask
    FL_pre = volFL.get_fdata() * mask
    T1_pre_nii = nib.Nifti1Image(T1_pre, volT1.affine)
    FL_pre_nii = nib.Nifti1Image(FL_pre, volFL.affine)
    T1_pre_nii.to_filename("Atlas/atlas{}_T1.nii.gz".format(i))
    FL_pre_nii.to_filename("Atlas/atlas{}_FL.nii.gz".format(i))
    shutil.copy("{}/atlas{}_mask.nii.gz".format(dir_name, i), "Atlas/atlas{}_mask.nii.gz".format(i))

Finalmente, selecionamos al azar un 20% de los pacientes para test y un 80% para train, que son guardados en carpetas separadas 'Atlas_train' y 'Atlas_test' respectivamente. Estas carpetas son las que se suben al directorio del drive ***IMAGENES***.

In [0]:
import random

n = int(0.2 * len(N))
random.seed(1000)

test = random.sample(N.tolist(), n)
train = list(set(N) - set(test))

for i in test:
    shutil.copy("Atlas/atlas{}_mask.nii.gz".format(i), "Atlas_test/atlas{}_mask.nii.gz".format(i))
    shutil.copy("Atlas/atlas{}_T1.nii.gz".format(i), "Atlas_test/atlas{}_T1.nii.gz".format(i))
    shutil.copy("Atlas/atlas{}_FL.nii.gz".format(i), "Atlas_test/atlas{}_FL.nii.gz".format(i))
    
for i in train:
    shutil.copy("Atlas/atlas{}_mask.nii.gz".format(i), "Atlas_train/atlas{}_mask.nii.gz".format(i))
    shutil.copy("Atlas/atlas{}_T1.nii.gz".format(i), "Atlas_train/atlas{}_T1.nii.gz".format(i))
    shutil.copy("Atlas/atlas{}_FL.nii.gz".format(i), "Atlas_train/atlas{}_FL.nii.gz".format(i))
    

# 2. Código FLEXCONN sin manipular

Utilizamos el código de FLEXCONN. Como el entrenamiento requiere bastantes horas, se realizarán checkpoins del código que serán guardados en drive ***TMF/FLEXCONN_CODE/Checkpoints***.

In [0]:
allow_growth = True

In [1]:
!pip uninstall  -y tensorflow

Uninstalling tensorflow-2.2.0rc2:
  Successfully uninstalled tensorflow-2.2.0rc2


Cambiamos el enterno a GPU.

In [2]:
!pip uninstall tensorflow-gpu

In [3]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 48kB/s 
     |████████████████████████████████| 419kB 36.3MB/s 
     |████████████████████████████████| 3.0MB 41.2MB/s 


In [5]:
import tensorflow as tf
print(tf.__version__)


2.0.0-alpha0


In [0]:
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()

In [7]:
tf.test.is_gpu_available()


True

## TPU

In [0]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf

In [0]:
import os
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [0]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
tpu_strategy.num_replicas_in_sync

Running on TPU  ['10.42.217.194:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.42.217.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.42.217.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


8

## TPU or GPU detection



In [0]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu, steps_per_run=128) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Conectamos *google-colab* con los archivos del Drive.



In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
!pip3 install -r "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/FLEXCONN_CODE/requirements.txt"

In [10]:
from __future__ import print_function, division

import argparse
import os
import re
import random
import time
import statsmodels.api as sm
import nibabel as nib
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import backend,losses
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import (AveragePooling2D, AveragePooling3D, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, concatenate, Lambda)
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.models import load_model
import sys
import copy
from tqdm import tqdm
from scipy import ndimage
from sklearn.mixture import GaussianMixture
from scipy.signal import argrelextrema

backend.set_floatx('float32')
backend.set_image_data_format('channels_last')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:

def multi_gpu_model(model, gpus):

    if isinstance(gpus, (list, tuple)):
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)
    
    def get_slice(data, num, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        in_shape = shape[1:]
        step = batch_size // parts
        if num == num_gpus - 1:
            size = batch_size - step * num
        else:
            size = step
        size = tf.concat([size, in_shape], axis=0)
        stride = tf.concat([step, in_shape * 0], axis=0)
        start = stride * num
        return tf.slice(data, start, size)
    
    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])
    
    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,
                                     output_shape=input_shape,
                                     arguments={'num': i,
                                                'parts': num_gpus})(x)
                    inputs.append(slice_i)
                
                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])
    
    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(concatenate(outputs,
                                      axis=0, name=name))
    return Model(model.inputs, merged)


def pad_image(vol, padsize):
    dim = vol.shape
    padsize = np.asarray(padsize, dtype=int)
    dim2 = dim + 2 * padsize
    temp = np.zeros(dim2, dtype=np.float32)
    temp[padsize:dim[0] + padsize, padsize:dim[1] + padsize, padsize:dim[2] + padsize] = vol
    return temp


def normalize_image(vol, contrast):
    # All MR images must be non-negative. Sometimes cubic interpolation may introduce negative numbers.
    # This will also affect if the image is CT, which not considered here. Non-negativity is required
    # while getting patches, where nonzero voxels are considered to collect patches.
    vol[vol<0] = 0
    temp = vol[np.nonzero(vol)].astype(float)
    q = np.percentile(temp, 99)
    temp = temp[temp <= q]
    temp = temp.reshape(-1, 1)
    bw = q / 80
    print("99th quantile is %.4f, gridsize = %.4f" % (q, bw))

    kde = sm.nonparametric.KDEUnivariate(temp)

    kde.fit(kernel='gau', bw=bw, gridsize=80, fft=True)
    x_mat = 100.0 * kde.density
    y_mat = kde.support

    indx = argrelextrema(x_mat, np.greater)
    indx = np.asarray(indx, dtype=int)
    heights = x_mat[indx][0]
    peaks = y_mat[indx][0]
    peak = 1.00
    print("%d peaks found." % (len(peaks)))


    if contrast.lower() == "t1" or contrast.lower() == "t1c":
        print("Double checking peaks with a GMM.")
        gmm = GaussianMixture(n_components=3, covariance_type='spherical', tol=0.001,
                reg_covar=1e-06, max_iter=100, n_init=1, init_params='kmeans', precisions_init=None,
                weights_init=(0.33, 0.33, 0.34), means_init=np.reshape((0.2 * q, 0.5 * q, 0.95 * q), (3, 1)),
                warm_start=False, verbose=1, verbose_interval=1)
        gmm.fit(temp.reshape(-1, 1))
        m = gmm.means_[2]
        peak = peaks[-1]
        if m / peak < 0.75 or m / peak > 1.25:
            print("WARNING: WM peak could be incorrect (%.4f vs %.4f). Please check." % (m, peak))
            peaks = m
        peak = peaks[-1]
        print("Peak found at %.4f for %s" % (peak, contrast))
    elif contrast.lower() in ['t2', 'pd', 'fl', 'flc']:
        peak_height = np.amax(heights)
        idx = np.where(heights == peak_height)
        peak = peaks[idx]
        print("Peak found at %.4f for %s" % (peak, contrast))
    else:
        print("Contrast must be either T1,T1C,T2,PD,FL, or FLC. You entered %s. Returning 1." % contrast)
    return vol/peak


def get_patches(vol4d, mask, opt):

    patchsize = opt['patchsize']
    nummodal = len(opt['modalities'])
    maxpatch = opt['max_patches']
    patchsize = np.asarray(patchsize, dtype=int)
    dsize = np.floor(patchsize / 2).astype(dtype=int)
    mask = np.asarray(mask, dtype=np.float32)
    rng = random.SystemRandom()

    if opt['loss'] == 'mse' or opt['loss'] == 'mae':
        if len(patchsize) == 3:
            blurmask = ndimage.filters.gaussian_filter(mask, sigma=(1, 1, 1))
        else:
            blurmask = np.zeros(mask.shape, dtype=np.float32)
            for t in range(0, mask.shape[2]):
                if np.ndarray.sum(mask[:, :, t]) > 0:
                    blurmask[:, :, t] = ndimage.filters.gaussian_filter(mask[:, :, t], sigma=(1, 1))

        blurmask = np.ndarray.astype(blurmask, dtype=np.float32)
        blurmask[blurmask < 0.0001] = 0
        blurmask = blurmask * 100  # Just to have reasonable looking error values during training, no other reason.
    else:
        blurmask = mask
    
    indx = np.nonzero(mask) # indx for positive patches
    indx = np.asarray(indx, dtype=int)


    num_patches = np.minimum(maxpatch, len(indx[0]))
    print('Number of patches used  = %d (out of %d, maximum %d)' % (num_patches, len(indx[0]), maxpatch))
    randindx = random.sample(range(0, len(indx[0])), num_patches)
    newindx = np.ndarray((3, num_patches))
    for i in range(0, num_patches):
        for j in range(0, 3):
            newindx[j, i] = indx[j, randindx[i]]
    newindx = np.asarray(newindx, dtype=int)

    # Add some negative samples as well
    r = 1 # Sampling ratio
    temp = copy.deepcopy(vol4d[:, :, :, 0])
    temp[temp > 0] = 1
    temp[temp <= 0] = 0
    temp = np.multiply(temp, 1 - mask)
    indx0 = np.nonzero(temp)
    indx0 = np.asarray(indx0, dtype=int)
    L = len(indx0[0])

    # Sample equal number of negative patches
    randindx0 = rng.sample(range(0, L), r * num_patches)
    newindx0 = np.ndarray((3, r * num_patches))
    for i in range(0, r * num_patches):
        for j in range(0, 3):
            newindx0[j, i] = indx0[j, randindx0[i]]
    newindx0 = np.asarray(newindx0, dtype=int)

    newindx = np.concatenate([newindx, newindx0], axis=1)


    if len(patchsize) == 2:
        matsize1 = ((r+1) * num_patches, patchsize[0], patchsize[1], 1)
        matsize2 = ((r+1) * num_patches, patchsize[0], patchsize[1], nummodal)
        
        image_patches = np.ndarray(matsize2, dtype=np.float32)
        mask_patches = np.ndarray(matsize1, dtype=np.float32)
        
        for i in range(0, (r+1)*num_patches):
            idx1 = newindx[0, i]
            idx2 = newindx[1, i]
            idx3 = newindx[2, i]
            
            for m in range(0, nummodal):
                image_patches[i, :, :, m] = vol4d[idx1 - dsize[0]:idx1 + dsize[0] + 1,
                                                  idx2 - dsize[1]:idx2 + dsize[1] + 1, idx3, m]
            mask_patches[i, :, :, 0] = blurmask[idx1 - dsize[0]:idx1 + dsize[0] + 1,
                                                idx2 - dsize[1]:idx2 + dsize[1] + 1, idx3]
    else:
        matsize1 = ((r+1)*num_patches, patchsize[0], patchsize[1], patchsize[2], 1)
        matsize2 = ((r+1)*num_patches, patchsize[0], patchsize[1], patchsize[2], nummodal)
        
        image_patches = np.ndarray(matsize2, dtype=np.float32)
        mask_patches = np.ndarray(matsize1, dtype=np.float32)
        
        for i in range(0, (r+1)*num_patches):
            idx1 = newindx[0, i]
            idx2 = newindx[1, i]
            idx3 = newindx[2, i]
            
            for m in range(0, nummodal):
                image_patches[i, :, :, :, m] = vol4d[idx1 - dsize[0]:idx1 + dsize[0] + 1,
                                                     idx2 - dsize[1]:idx2 + dsize[1] + 1,
                                                     idx3 - dsize[2]:idx3 + dsize[2] + 1, m]
            mask_patches[i, :, :, :, 0] = blurmask[idx1 - dsize[0]:idx1 + dsize[0] + 1,
                                                   idx2 - dsize[1]:idx2 + dsize[1] + 1,
                                                   idx3 - dsize[2]:idx3 + dsize[2] + 1]
    return image_patches, mask_patches

def check_nifti_filepath(directory, file_prefix):
    filepath = os.path.join(directory, file_prefix + '.nii.gz')
    filepath = filepath if os.path.exists(filepath) else os.path.join(directory, file_prefix + '.nii')
    if not os.path.exists(filepath):
        raise ValueError('File %s does not exists' % filepath)
    return filepath


In [0]:

def get_inception_3d(inlayer, base_filters):
    conv_inception1a = Conv3D(base_filters * 4, (1, 1, 1), activation='relu', padding='same')(inlayer)
    
    conv_inception2a = Conv3D(base_filters * 6, (1, 1, 1), activation='relu', padding='same')(inlayer)
    conv_inception4a = Conv3D(base_filters * 8, (3, 3, 3), activation='relu', padding='same')(conv_inception2a)
    
    conv_inception3a = Conv3D(base_filters, (1, 1, 1), activation='relu', padding='same')(inlayer)
    conv_inception5a = Conv3D(base_filters * 2, (5, 5, 5), activation='relu', padding='same')(conv_inception3a)
    
    pool_inception1a = AveragePooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(inlayer)
    conv_inception6a = Conv3D(base_filters * 2, (1, 1, 1), activation='relu', padding='same')(pool_inception1a)
    
    pool_inception2a = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(inlayer)
    conv_inception7a = Conv3D(base_filters * 2, (1, 1, 1), activation='relu', padding='same')(pool_inception2a)
    print('concat')
    outlayer = concatenate([conv_inception1a, conv_inception4a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception5a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception6a], axis=-1)
    outlayer = concatenate([outlayer, conv_inception7a], axis=-1)
    return outlayer


def get_model_3d(base_filters, numchannel, gpus,loss):
    inputs = Input((None, None, None, int(numchannel)))
    conv1 = Conv3D(base_filters * 8, (3, 3, 3), activation='relu', padding='same', strides=(1, 1, 1))(inputs)
    conv2 = Conv3D(base_filters * 8, (3, 3, 3), activation='relu', padding='same', strides=(1, 1, 1))(conv1)
    print('first inception')
    
    inception1 = get_inception_3d(conv2, base_filters)
    inception2 = get_inception_3d(inception1, base_filters)
    inception3 = get_inception_3d(inception2, base_filters)
    
    convconcat1 = Conv3D(base_filters * 4, (3, 3, 3), activation='relu', padding='same', strides=(1, 1, 1))(inception3)
    convconcat2 = Conv3D(base_filters * 4, (3, 3, 3), activation='relu', padding='same', strides=(1, 1, 1))(convconcat1)
    print('check loss')
    if loss == 'bce':
        conv_last = Conv3D(1, (3, 3, 3), activation='sigmoid', padding='same', strides=(1, 1, 1))(convconcat2)
    else:
        conv_last = Conv3D(1, (3, 3, 3), activation='relu', padding='same', strides=(1, 1, 1))(convconcat2)
    model = Model(inputs=inputs, outputs=conv_last)
    print(gpus)
    if (gpus) > 1:
        #import tensorflow as tf
        with tf.device('/cpu:0'):
            model = multi_gpu_model(model, gpus)
    print('compile model')
    if loss == 'bce':
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    elif loss == 'mae':
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_absolute_error')
    else:
        model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
    return model


In [0]:

def main(kwargs):
    
    nummodal = int(len(kwargs['modalities']))
    padsize = np.max(np.array(kwargs['patchsize']) + 1) / 2
    kwargs['batchsize'] = kwargs['batchsize'] * kwargs['numgpu']
    f = 0 # num_patches
    #--efm
    atlas = os.listdir(kwargs['atlasdir'])
    nums = []
    for im in atlas:
        nums.append(re.findall('\d+', im)[0])
    N = np.unique(nums)
    #---

    for i in N:
        maskname = check_nifti_filepath(kwargs['atlasdir'], ('atlas%s' % (i)) + '_' + 'mask')
        f += min(kwargs['max_patches'], nib.load(maskname).get_fdata().sum())
    
    print('Total number of lesion patches = ' + str(int(f)))
    r = 1 # ratio between positive and negative patches
    mask_patches = np.zeros((int((r+1)*f),) + kwargs['patchsize'] + (1,), dtype=np.float32)
    image_patches = np.zeros((int((r+1)*f),) + kwargs['patchsize'] + (nummodal,), dtype=np.float32)
    
    time_id = time.strftime('%d-%m-%Y_%H-%M-%S')
    print('Unique ID is %s ' % time_id)

    con = '+'.join([str(mod).upper() for mod in kwargs['modalities']])
    psize = 'x'.join([str(side) for side in kwargs['patchsize']])
    outname = 'FLEXCONN_Model_' + psize + '_Orient%d%d%d_' + con + '_' + time_id + '.h5'
    
    codes = [(0, 1, 2), (1, 2, 0), (2, 0, 1)]
    
    for orient in range(1 if kwargs['axial_only'] else 3):
        transpose_code = codes[orient]
        orient_outname = os.path.join(kwargs['outdir'], outname % transpose_code)
        if kwargs['loss'] == 'bce':
            tempoutname = orient_outname.replace('.h5', '_epoch-{epoch:03d}_acc-{val_accuracy:.4f}.h5')
        else:
            tempoutname = orient_outname.replace('.h5', '_epoch-{epoch:03d}_val_loss-{val_loss:.4f}.h5')

        print('Model for orientation %s will be written at %s' % (str(transpose_code), orient_outname))

        # Re-initialize total matrix size because some of them could be discarded during multi-gpu process
        mask_patches = np.zeros((int((r+1) * f),) + kwargs['patchsize'] + (1,), dtype=np.float32)
        image_patches = np.zeros((int((r+1) * f),) + kwargs['patchsize'] + (nummodal,), dtype=np.float32)

        patch_count = 0
        #---efm
        for i in N: #---
            #segpath = check_nifti_filepath(kwargs['atlasdir'], ('atlas%02d' % (i + 1)) + '_' + 'lesion')
            segpath = check_nifti_filepath(kwargs['atlasdir'], ('atlas%s' % (i)) + '_' + 'mask') #---efm---
            mask = np.transpose(pad_image(nib.load(segpath).get_fdata(), padsize),
                                axes=transpose_code).astype(np.float32)
            vol4d = np.zeros(mask.shape + (nummodal,), dtype=np.float32)
            
            # loading volume & normalize (T1 & FL)
            for j in range(nummodal):
                filepath = check_nifti_filepath(kwargs['atlasdir'],
                                                ('atlas%s' % (i)) + '_' + kwargs['modalities'][j].upper())
                #                                ('atlas%02d' % (i + 1)) + '_' + kwargs['modalities'][j].lower())
                print('Reading %s' % filepath)
                vol4d[:, :, :, j] = np.transpose(pad_image(normalize_image(nib.load(filepath).get_fdata(),kwargs['modalities'][j]), padsize),
                                                 axes=transpose_code).astype(np.float32)
            print('Atlas %s size = %d x %d x %d x %d ' % ((i,) + vol4d.shape))

            # Get 4D patches size
            image_patches_a, mask_patches_a = get_patches(vol4d, mask, kwargs)
            num_patches = image_patches_a.shape[0]
            print('Atlas %s : indices [%d,%d)' % (i, patch_count, patch_count + num_patches))
            image_patches[patch_count:patch_count + num_patches, :, :, :] = image_patches_a
            mask_patches[patch_count:patch_count + num_patches, :, :, :] = mask_patches_a
            patch_count += num_patches
            print('-' * 100)

        image_patches = image_patches[0:patch_count, :, :, :]
        mask_patches = mask_patches[0:patch_count, :, :, :]

        """
        If the number of patches within training & validation sets are not multiple of number of GPU,
        there could be some arbitrary CUDNN error. Although ideally this should be taken care within
        multi_gpu_model function, it is not; discarding last couple of samples is easiest.
        This will not work if the number of samples is low, e.g. marmosets or CT. Use single gpu in those cases.
        """
        numgpu = kwargs['numgpu']
        if numgpu>1:
            L = image_patches.shape[0]
            L = np.floor(np.floor(L * 0.2) / numgpu) * numgpu * 5
            L = np.asarray(L, dtype=int)
            image_patches = image_patches[0:L, :, :, :]
            mask_patches = mask_patches[0:L, :, :, :]

        print('Total number of patches collected = ' + str(patch_count))
        print('Sizes of the input matrices are ' + str(image_patches.shape) + ' and ' + str(mask_patches.shape))

        # if len(kwargs['patchsize']) == 2:
        #     model = get_model_2d(kwargs['base_filters'], nummodal, kwargs['numgpu'], kwargs['loss'])
        # else:
        #     model = get_model_3d(kwargs['base_filters'], nummodal, kwargs['numgpu'], kwargs['loss'])
        print(nummodal)
        model = get_model_3d(kwargs['base_filters'], nummodal, kwargs['numgpu'], kwargs['loss'])
        print('model 3d created')

        if kwargs['loss'] == 'bce':
            callbacks = [ModelCheckpoint(tempoutname, monitor='val_accuracy', verbose=1, save_best_only=True,
                                     period=kwargs['period'], mode='max')] if kwargs['period'] > 0 else None
            dlr = ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=2,
                                mode='max', verbose=1, cooldown=2, min_lr=1e-8)
            earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.0002, patience=5,
                                  verbose=1, mode='max')
        else:

            callbacks = [ModelCheckpoint(tempoutname, monitor='val_loss', verbose=1, save_best_only=True,
                                         period=kwargs['period'], mode='min')] if kwargs['period'] > 0 else None
            dlr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,
                                    mode='min', verbose=1, cooldown=2, min_lr=1e-8)
            earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=10,
                                      verbose=1, mode='min')
        callbacks.append(dlr)
        callbacks.append(earlystop)

        if kwargs['initmodel'] != 'None' and os.path.exists(kwargs['initmodel']):
            dict = {"tf": tf,
                    }
            oldmodel = load_model(kwargs['initmodel'],custom_objects=dict)
            model.set_weights(oldmodel.get_weights())
            print("Initializing from existing model %s" % (kwargs['initmodel']))
        # efm
        # image_patches = np.expand_dims(image_patches, axis = 1)
        model.fit(image_patches, mask_patches, batch_size=kwargs['batchsize'], epochs=kwargs['epoch'], verbose=1,
                  validation_split=0.2, callbacks=callbacks, shuffle=True)
        # shuffle = True should be used, but sometimes that results in instability. More experimentation needed
        
        print('Final model is written at ' + orient_outname)
        # if len(kwargs['gpu_ids']) > 1:
        #     """
        #     Training with multi-gpu model has a bug where the trained model can not be loaded to a single gpu for
        #     testing, giving some zero batch size errors, such as
        #         1) could not convert BatchDescriptor {count: 0 ...
        #         2) cudnn tensor descriptor: CUDNN_STATUS_BAD_PARAM
        #     Therefore the following fix works when models trained with multi-gpu are used for single gpu.
        #     It is obtained from the following link,
        #        https://stackoverflow.com/questions/47210811/can-not-save-model-using-model-save-following-multi-
        #        gpu-model-in-keras/48066771#48066771
        #     ** This is fixed in Keras 2.1.5, although it is better for backwards compatibility **
        #     ** Don't use default multi_gpu_model from Keras 2.1.5, sometimes the resulting model file can't be read. 
        #     Not sure why.**
        #     """

        #     with tf.device("/cpu:0"):
        #         single_model = get_model_2d(kwargs['base_filters'], nummodal, 1, kwargs['loss']) if len(kwargs['patchsize']) == 2 \
        #             else get_model_3d(kwargs['base_filters'], nummodal, 1, kwargs['loss'])
        #     if kwargs['loss'] == 'bce':
        #         single_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
        #     else:
        #         single_model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
        #     single_model.set_weights(model.get_weights())
        #     single_model.save(filepath=orient_outname)
        # else:
        model.save(filepath=orient_outname,  overwrite=True, include_optimizer=True)


In [0]:
natlas = 6
outdir = "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/FLEXCONN_CODE/Checkpoints"
modalities = ['T1', 'FL']
psize = [100,100,100]
atlasdir = "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/IMAGES/AT"
batchsize = 64
epoch = 50
save = 1
axialonly = False 
basefilters = 8
maxpatches = 50000
gpuids = None
gpu_ids = None
numgpu = 1
loss = 'bce'
INITMODEL = None

In [0]:
if gpuids is not None:
    gpu_ids = gpuids
else:
    gpu_ids = range(numgpu)

numgpu = len(gpu_ids)
batchsize = (batchsize // numgpu) * numgpu



In [0]:
kwargs = { 'numatlas': natlas,
        'outdir': outdir,
        'modalities': [item.upper() for item in modalities],
        'patchsize': tuple(psize),
        'atlasdir': atlasdir,
        'batchsize': batchsize,
        'epoch': epoch,
        'period': save,
        'axial_only': axialonly,
        'base_filters': basefilters,
        'max_patches': maxpatches,
        'gpu_ids': 1,
        'numgpu': 1,
        'loss': str(loss).lower(),
        'initmodel': INITMODEL,
        }

In [0]:
nummodal = int(len(kwargs['modalities']))
padsize = np.max(np.array(kwargs['patchsize']) + 1) / 2
kwargs['batchsize'] = kwargs['batchsize'] * kwargs['numgpu']
f = 0 # num_patches
#--efm
atlas = os.listdir(kwargs['atlasdir'])
nums = []
for im in atlas:
    nums.append(re.findall('\d+', im)[0])
N = np.unique(nums)
#---

In [18]:

for i in N:
    maskname = check_nifti_filepath(kwargs['atlasdir'], ('atlas%s' % (i)) + '_' + 'mask')
    f += min(kwargs['max_patches'], nib.load(maskname).get_fdata().sum())
print('Total number of lesion patches = ' + str(int(f)))

Total number of lesion patches = 20429


In [19]:
r = 1 # ratio between positive and negative patches
mask_patches = tf.zeros((int((r+1)*f),) + kwargs['patchsize'] + (1,), dtype=tf.dtypes.float32)

ResourceExhaustedError: ignored

In [0]:

image_patches = np.zeros((int((r+1)*f),) + kwargs['patchsize'] + (nummodal,), dtype=np.float32)

time_id = time.strftime('%d-%m-%Y_%H-%M-%S')
print('Unique ID is %s ' % time_id)

In [0]:


r = 1 # ratio between positive and negative patches
mask_patches = np.zeros((int((r+1)*f),) + kwargs['patchsize'] + (1,), dtype=np.float32)
image_patches = np.zeros((int((r+1)*f),) + kwargs['patchsize'] + (nummodal,), dtype=np.float32)

time_id = time.strftime('%d-%m-%Y_%H-%M-%S')
print('Unique ID is %s ' % time_id)

con = '+'.join([str(mod).upper() for mod in kwargs['modalities']])
psize = 'x'.join([str(side) for side in kwargs['patchsize']])
outname = 'FLEXCONN_Model_' + psize + '_Orient%d%d%d_' + con + '_' + time_id + '.h5'

codes = [(0, 1, 2), (1, 2, 0), (2, 0, 1)]

In [0]:
# Patch size must be odd
for i in range(len(psize)):
    psize[i] = (psize[i]//2)*2 + 1

if INITMODEL is None:
    INITMODEL = 'None'

opt = { 'numatlas': natlas,
        'outdir': outdir,
        'modalities': [item.upper() for item in modalities],
        'patchsize': tuple(psize),
        'atlasdir': atlasdir,
        'batchsize': batchsize,
        'epoch': epoch,
        'period': save,
        'axial_only': axialonly,
        'base_filters': basefilters,
        'max_patches': maxpatches,
        'gpu_ids': 1,
        'numgpu': 1,
        'loss': str(loss).lower(),
        'initmodel': INITMODEL,
        }

main(opt)

Total number of lesion patches = 20429


In [0]:
models = "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/FLEXCONN_CODE/Models/*.h5"
inputdir = "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/IMAGES/Atlas_train"
imgs = ["atlas12_T1.nii.gz","atlas12_FL.nii.gz","atlas13_T1.nii.gz", "atlas13_FL.nii.gz","atlas16_T1.nii.gz", "atlas16_FL.nii.gz"]
modalities = ['T1', 'FL']
psize = [100,100]
outdir = "/content/gdrive/My Drive/UOC/TFM-Elisa-2020/FLEXCONN_CODE/Results"
gpu = 1
threshold = 0.5
